<a href="https://colab.research.google.com/github/Aggregate-Intellect/xir/blob/main/wikiQA_sentence_level_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beir
!pip install tensorflow-text
!pip install farm-haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install tensorflow

In [2]:
import pandas as pd

import nltk.data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# wikiQA dataset

In [3]:
from datasets import load_dataset

wikiqa_data = load_dataset(
    'wiki_qa',
    split='train',
)
next(iter(wikiqa_data))

Using custom data configuration default


Generating test split:   0%|          | 0/6165 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2733 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20360 [00:00<?, ? examples/s]

Dataset wiki_qa downloaded and prepared to /root/.cache/huggingface/datasets/wiki_qa/default/0.1.0/d2d236b5cbdc6fbdab45d168b4d678a002e06ddea3525733a24558150585951c. Subsequent calls will reuse this data.


{'answer': 'A partly submerged glacier cave on Perito Moreno Glacier .',
 'document_title': 'Glacier cave',
 'label': 0,
 'question': 'how are glacier caves formed?',
 'question_id': 'Q1'}

In [4]:
pd_data = pd.DataFrame(wikiqa_data)
pd_data.to_csv("wikiQA_dataset.csv", index=False)
pd_data

,question_id,question,document_title,answer,label
0,Q1,how are glacier caves formed?,Glacier cave,A partly submerged glacier cave on Perito More...,0
1,Q1,how are glacier caves formed?,Glacier cave,The ice facade is approximately 60 m high,0
2,Q1,how are glacier caves formed?,Glacier cave,Ice formations in the Titlis glacier cave,0
3,Q1,how are glacier caves formed?,Glacier cave,A glacier cave is a cave formed within the ice...,1
4,Q1,how are glacier caves formed?,Glacier cave,"Glacier caves are often called ice caves , but...",0
...,...,...,...,...,...
20355,Q3043,what is section eight housing,Section 8 (housing),A tenant who leaves a subsidized project will ...,0
20356,Q3043,what is section eight housing,Section 8 (housing),The United States Department of Housing and Ur...,0
20357,Q3044,what is the main type of restaurant,Category:Types of restaurants,Restaurants categorized by type and informatio...,0
20358,Q3046,what is us dollar worth based on,History of the United States dollar,U.S. Federal Reserve notes in the mid-1990s,0


In [5]:
total_passage_number = len(pd_data['document_title'].unique().tolist())
print(total_passage_number)

total_question_number = len(pd_data['question'].unique().tolist())
print(total_question_number)

1995
2118


In [6]:
processed_wikiqa = pd.DataFrame(columns = ["question_id", "question", "document_title", "document_content", "answer"])

for question in pd_data['question'].unique().tolist():
    pd_question = pd_data.loc[pd_data['question'] == question]   
    content = ' '.join(pd_question["answer"].tolist())
    pd_answer = pd_question[pd_question['label'] == 1]
    if not pd_answer.empty:
        answer = ' '.join(pd_answer['answer'].tolist())
        new_row = {"question_id": pd_question["question_id"].iloc[0], 
                  "question" : pd_question["question"].iloc[0],
                  "document_title" : pd_question["document_title"].iloc[0],
                  "document_content" : content,
                  "answer": answer}
        # print(pd_answer['question_id'].iloc[0])
        # if pd_answer['question_id'].iloc[0] == 'Q1537':
        #     pd_answer
        processed_wikiqa = processed_wikiqa.append(new_row, ignore_index=True)
processed_wikiqa.to_csv("processed_wikiqa.csv", index=False)
processed_wikiqa

,question_id,question,document_title,document_content,answer
0,Q1,how are glacier caves formed?,Glacier cave,A partly submerged glacier cave on Perito More...,A glacier cave is a cave formed within the ice...
1,Q16,how much is 1 tablespoon of water,Tablespoon,This tablespoon has a capacity of about 15 mL....,This tablespoon has a capacity of about 15 mL....
2,Q17,how much are the harry potter movies worth,Harry Potter,Harry Potter is a series of seven fantasy nove...,The series also originated much tie-in merchan...
3,Q18,how a rocket engine works,Rocket engine,RS-68 being tested at NASA's Stennis Space Cen...,"A rocket engine, or simply ""rocket"", is a jet ..."
4,Q21,how are cholera and typhus transmitted and pre...,Cholera,Cholera is an infection in the small intestine...,Transmission occurs primarily by drinking wate...
...,...,...,...,...,...
868,Q3034,what is melissa and joey about,Melissa & Joey,Melissa & Joey is an ABC Family original telev...,The series follows local politician Mel Burke ...
869,Q3037,What is an economic feature?,Economics,Economics is the social science that analyzes ...,Other broad distinctions include those between...
870,Q3039,what is the average american income,Household income in the United States,US county household median income 2009. Median...,"U.S. median household income fell from $51,144..."
871,Q3042,When was Apple Computer founded,Apple Inc.,"Apple Inc., formerly Apple Computer, Inc., is ...","The company was founded on April 1, 1976, and ..."


# Faiss Document Store

In [7]:
from typing import List
import requests
import pandas as pd
from haystack import Document
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import RAGenerator, DensePassageRetriever
from haystack.utils import fetch_archive_from_http, print_answers, print_documents

from haystack.document_stores import FAISSDocumentStore
from haystack import Document

In [8]:
docs = []
for index, row in processed_wikiqa.iterrows():
    # create haystack document object with text content and doc metadata
    doc = Document(
        content = row['document_content'],
        meta = {
            "document_title": row['document_title']
        }
    )
    docs.append(doc)

document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)

# Delete existing documents in documents store
document_store.delete_documents()

document_store.write_documents(docs)
document_store.get_document_count()

INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


Writing Documents:   0%|          | 0/873 [00:00<?, ?it/s]

852

# Passage Retriever

In [9]:
from haystack.retriever.dense import EmbeddingRetriever

retriever = EmbeddingRetriever(
   document_store=document_store,
   embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
   model_format="sentence_transformers"
)

document_store.update_embeddings(retriever, batch_size=128)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.retriever.dense -  Init retriever using embeddings of model flax-sentence-embeddings/all_datasets_v3_mpnet-base


Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/591 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
document_store.get_embedding_count()

852

# Document Search Pipeline

In [11]:
from haystack.pipelines import DocumentSearchPipeline
pipeline = DocumentSearchPipeline(retriever = retriever)

# Calculate Accuracy

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

number_correctly_retrieved_passage = 0
number_correctly_retrieved_sentence = 0
number_documents = 852


for index, row in processed_wikiqa.iterrows():
    query = row['question']
    result = pipeline.run(
             query=query, 
             params={"Retriever": {"top_k": 1}}
             )
    retrieved_document = result["documents"][0].content
    target_document = processed_wikiqa['document_content'][index]
    
    if retrieved_document == target_document:
        number_correctly_retrieved_passage += 1
        
        sentence_document_store = InMemoryDocumentStore()
        sentence_document_store.delete_documents()
        docs = []
        sentence_list = nltk.tokenize.sent_tokenize(retrieved_document)
        print(sentence_list)
        for sentence in sentence_list:
            doc = Document(content=sentence,
                           meta={"article_title": processed_wikiqa['document_title']})
            docs.append(doc)
            sentence_document_store.write_documents(docs)
        
        sentence_retriever = EmbeddingRetriever(
            document_store=sentence_document_store,
            embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
            model_format="sentence_transformers")
        
        sentence_document_store.update_embeddings(sentence_retriever)
        sentence_pipeline = DocumentSearchPipeline(retriever = sentence_retriever)
        
        sentence_result = sentence_pipeline.run(query=query, 
                                                params={"Retriever": {"top_k": 1}})
        retrieved_sentence = sentence_result["documents"][0].content
        target_sentence = processed_wikiqa['answer'][index]
        print(str(retrieved_sentence))
        print(" ".join(str(target_sentence).split()))
        if str(retrieved_sentence) in " ".join(str(target_sentence).split()):
            number_correctly_retrieved_sentence += 1
            print("Found")
    # else:
    #     print("============================")
    #     print("question id: ", processed_wikiqa['question_id'][index])
    #     print("question: ", processed_wikiqa['question'][index])
    #     print("retrieved document: ", retrieved_document)
    #     print("target docment: ", target_document)
    if index == 851:
        break

In [14]:
# result of first 300 query
print("Passage Retriever Accuracy: ", number_correctly_retrieved_passage / 852)
print("Sentence Retriever Accuracy: ", number_correctly_retrieved_sentence / number_correctly_retrieved_passage)

Passage Retriever Accuracy:  0.9800469483568075
Sentence Retriever Accuracy:  0.61437125748503
